# Plot groups' analysis
## Set parameters

In [ ]:
import os

CONFIG_FILE_NAME = "braian_config.toml"                     # assumes the file is in DATA_ROOT directory
# USE_REMOTE_DATA -> if True, it tries to read the data on the laboratory's server
# EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "p6", False
# EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "experiment", False
# EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "proof", False
EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "rebased_on_mjd", False
EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "Cariplo_NRe/IEGs Experiment", True

# ###################################### REMOTE DIRECTORIES #####################################
IS_COLLABORATION_PROJ = False
COLLABORATION_DIRECTORY = os.path.join("Mathias Schmidt", "soumnya")

# ###################################### LOCAL DIRECTORIES ######################################
# DATA_ROOT  = f"../data/experiments/sowmya/{EXPERIMENT_DIRECTORY}"
# PLOTS_ROOT = f"../plots/sowmya/{EXPERIMENT_DIRECTORY}"
DATA_ROOT  = f"../data/experiments/{EXPERIMENT_DIRECTORY}"
PLOTS_ROOT = f"../plots/{EXPERIMENT_DIRECTORY}"

In [ ]:
# on which comparison of CONFIG_FILE_NAME to run the analysis
COMPARISON_ID = 1

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
SAVED_PLOT_EXTENSION = ".html"                              # '.html' for interactive plot
                                                            # '.svg' for vectorized image
                                                            # '.png'/'.jpg'/... for rasterized image

# ########################################## PIE CHART ##########################################
PIE_SAVE_PLOT = True
PIE_SHOW_PLOT = True
PIE_USE_ACRONYMS = False
PIE_HOLE = 0.4                                              # a value between 0 (no hole) and 1 (just a hole, no plot)
PIE_TEXT_SIZE = 25

# ########################################## BAR PLOT ###########################################
BAR_SAVE_PLOT = True
BAR_SHOW_PLOT = False
BAR_ANIMAL_SIZE = 8
BAR_TITLE_TEXT_SIZE = 40
BAR_AXIS_TEXT_SIZE = 22
BAR_HEIGHT = 30
BAR_WIDTH = 1_500
BAR_TITLE = ""
BAR_USE_ACRONYMS = True

## Script's code
run all cell below

In [ ]:
import sys
from plotly.colors import DEFAULT_PLOTLY_COLORS

project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
if USE_REMOTE_DATA:
    DATA_ROOT, PLOTS_ROOT = BraiAn.remote_dirs(EXPERIMENT_DIRECTORY, IS_COLLABORATION_PROJ, COLLABORATION_DIRECTORY)

config_file = os.path.join(DATA_ROOT, CONFIG_FILE_NAME)

In [ ]:
import tomllib

with open(config_file, "rb") as f:
    config = tomllib.load(f)
config
# ######################################### LOAD CONFIG #########################################
EXPERIMENT_NAME = config["experiment"]["name"]

ATLAS_VERSION = config["atlas"]["version"]
BRANCHES_TO_EXCLUDE = config["atlas"]["excluded-branches"]

NORMALIZATION = config["brains"]["normalization"]
MIN_AREA = config["comparison"]["min-area"]                                 # area in mm². If a region of one animal is smaller, that same animal won't be displayed in the plots
REGIONS_TO_PLOT_SELECTION_METHOD = config["comparison"]["regions-to-plot"]  # Available options are:
                                                                            #   - "summary structures"
                                                                            #   - "major divisions"
                                                                            #   - "depth <n>" where <n> is an integer of the depth desired
                                                                            #   - "structural level <n>" where <n> is an integer of the level desired
                                                                            #   - "pls <experiment> <salience_threshold>" (e.g., "pls proof 1.2")
                                                                            # where <n> is an integer of the depth/level desired

from collections import namedtuple
GroupDirectory = namedtuple("GroupDirectory", "id name dirs")
groups = [
    GroupDirectory(
        id=int(group[len("group"):])-1,
        name=config["experiment"][group]["name"],
        dirs=config["experiment"][group]["dirs"]
    ) for group in config["experiment"] if group.startswith("group") and group[len("group"):].isdigit()
]

comparison_groups = config["comparison"][str(COMPARISON_ID)]["groups"]
Comparison = namedtuple("Comparison", "groups dir")
comparison = Comparison(
    groups=[groups[id-1] for id in comparison_groups],
    dir=config["comparison"][str(COMPARISON_ID)]["dir"]
)

In [ ]:
data_input_path     = os.path.join(DATA_ROOT, "BraiAn_output")
data_output_path    = os.path.join(data_input_path, comparison.dir)
plots_output_path   = os.path.join(PLOTS_ROOT, comparison.dir)

if not(os.path.exists(data_output_path)):
    os.makedirs(data_output_path, exist_ok=True)

if not(os.path.exists(plots_output_path)):
    os.makedirs(plots_output_path, exist_ok=True)

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
BraiAn.cache(path_to_allen_json, "http://api.brain-map.org/api/v2/structure_graph_download/1.json")
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, BRANCHES_TO_EXCLUDE, version=ATLAS_VERSION)

In [ ]:
animal_groups: list[BraiAn.AnimalGroup] = []
for group in comparison.groups:
    animal_group = BraiAn.AnimalGroup.from_csv(group.name, data_input_path, f"cell_counts_{group.name}_{NORMALIZATION.lower()}.csv")
    animal_groups.append(animal_group)
    print(f"Group '{animal_group.name}' - #animals: {animal_group.n}, marker: {animal_group.markers}")


In [ ]:
match REGIONS_TO_PLOT_SELECTION_METHOD:
    case "summary structures":
        # selects the Summary Strucutures
        path_to_summary_structures = os.path.join(project_path, "data", "AllenSummaryStructures.csv")
        AllenBrain.select_from_csv(path_to_summary_structures)
    case "major divisions":
        AllenBrain.select_regions(BraiAn.MAJOR_DIVISIONS)
    case "smallest":
        AllenBrain.select_leaves()
    case s if s.startswith("pls"):
        options = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")
        assert len(options) == 3, "The 'REGIONS_TO_PLOT_SELECTION_METHOD' option is invalid. Make sure it follows the follows the following pattern: \"pls <experiment> <salience_threshold>\" (e.g., \"pls proof 1.2\")"
        pls_experiment, pls_threshold = options[1:]
        pls_threshold = pls_threshold.replace(".", "_")
        assert len(animal_groups) == 2, f"You can't use the PLS of '{pls_experiment}' for selecting the regions to plot because '{comparison.dir}' has too many groups ({len(animal_groups)})"
        pls_file = f"pls_{animal_groups[0].markers[0]}_{NORMALIZATION}_salient_regions_above_{pls_threshold}.csv".lower()
        regions_to_plot_pls_csv = os.path.abspath(os.path.join(DATA_ROOT, os.pardir, pls_experiment, "BraiAn_output", comparison.dir, pls_file))
        assert os.path.isfile(regions_to_plot_pls_csv), f"Could not find the file '{regions_to_plot_pls_csv}'"
        AllenBrain.select_from_csv(regions_to_plot_pls_csv, key="acronym")
    case s if s.startswith("depth"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            depth = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'depth' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        AllenBrain.select_at_depth(depth)
    case s if s.startswith("structural level"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            level = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'structural level' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        AllenBrain.select_at_structural_level(level)
    case _:
        raise Exception(f"Invalid value '{REGIONS_TO_PLOT_SELECTION_METHOD}' for REGIONS_TO_PLOT_SELECTION_METHOD")
regions_to_plot = AllenBrain.get_selected_regions()
print(f"You selected {len(regions_to_plot)} regions to plot.")

In [ ]:
for animal_group in animal_groups:
    animal_group.remove_smaller_subregions(MIN_AREA, AllenBrain)

In [ ]:
regions_to_plot_selection_method_str = REGIONS_TO_PLOT_SELECTION_METHOD.replace(".", "_").replace(" ", "_")

if len(animal_groups) == 2:
    for g1_marker, g2_marker in zip(animal_groups[0].markers, animal_groups[1].markers):
        prism_data = BraiAn.as_prism_data(NORMALIZATION, *animal_groups, AllenBrain, g1_marker=g1_marker, g2_marker=g2_marker)
        prism_data = prism_data.loc[regions_to_plot]
        prism_file = f"prism_{comparison.dir}_{animal_groups[0].markers[0]}_{NORMALIZATION}_{regions_to_plot_selection_method_str}.csv"
        BraiAn.save_csv(prism_data.swaplevel(), data_output_path, prism_file.lower(), sep=",", overwrite=True)

In [ ]:
fig = BraiAn.plot_pie(regions_to_plot, AllenBrain, use_acronyms=PIE_USE_ACRONYMS, hole=PIE_HOLE, line_width=1, text_size=PIE_TEXT_SIZE)
if PIE_SAVE_PLOT:
    plot_filename = f"piechart_{comparison.dir}_{animal_groups[0].markers[0]}_{NORMALIZATION}_{regions_to_plot_selection_method_str}{SAVED_PLOT_EXTENSION}".lower()
    plot_filepath = os.path.join(plots_output_path, plot_filename)
    match SAVED_PLOT_EXTENSION.lower():
        case ".html":
            fig.write_html(plot_filepath, config=dict(toImageButtonOptions=dict(format="svg")))
        case _:
            fig.write_image(plot_filepath)
if PIE_SHOW_PLOT:
    fig.show()

In [ ]:
for marker in animal_groups[0].markers:
    fig = BraiAn.plot_groups(AllenBrain, *animal_groups, selected_regions=regions_to_plot, marker=marker,
                                plot_title=BAR_TITLE, title_size=BAR_TITLE_TEXT_SIZE, axis_size=BAR_AXIS_TEXT_SIZE, animal_size=BAR_ANIMAL_SIZE,
                                use_acronyms=BAR_USE_ACRONYMS, colors=(DEFAULT_PLOTLY_COLORS[group.id] for group in comparison.groups),
                                width=BAR_WIDTH, barheight=BAR_HEIGHT, bargap=0.3, bargroupgap=0.0)

    if BAR_SAVE_PLOT:
        plot_filename = f"barplot_{comparison.dir}_{marker}_{NORMALIZATION}_{regions_to_plot_selection_method_str}{SAVED_PLOT_EXTENSION}".lower()
        plot_filepath = os.path.join(plots_output_path, plot_filename)
        match SAVED_PLOT_EXTENSION.lower():
            case ".html":
                fig.write_html(plot_filepath, config=dict(toImageButtonOptions=dict(format="svg")))
            case _:
                fig.write_image(plot_filepath)
    if BAR_SHOW_PLOT:
        fig.show()

In [ ]:
import importlib
__imported_modules = sys.modules.copy()
for module_name, module in __imported_modules.items():
    if not module_name.startswith("BraiAn"):
        continue
    try:
        importlib.reload(module)
    except ModuleNotFoundError:
        continue